# FINAL PROJECT SIC - CHATBOT HỖ TRỢ TRA CỨU THÔNG TIN TRƯỜNG ĐH SƯ PHẠM KỸ THUẬT
- Author: Dang Kim Thanh - AI 1

# III. Load model and answer question

## Import needed lib

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.ctransformers import CTransformers
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate

## Load quantization model

In [ ]:
model_name = "model/vinallama-7b-chat_q5_0.gguf"

def load_model():
    llm = CTransformers(
        model = model_name,
        model_type = 'llama',
        max_new_token = 1024,
        temperature = 0.2
    )
    return llm

## Or Load original model

In [18]:
# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False # To quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 4, 4
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}

# transformed_dataset is there

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

finetune_model = "model/finetune_model.pt"
tokenizer_path = "model/tokenizer/"
# finetune_model = "/content/drive/MyDrive/SIC_AI/model/finetune_model.pt"
# tokenizer_path = "/content/drive/MyDrive/SIC_AI/model/tokenizer"

def load_model():
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        finetune_model,
        quantization_config=bnb_config,
        device_map=device_map
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Load LLaMA tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

    # Create a text generation pipeline
    llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

    return llm

## Prompt Template and QA Question

In [19]:
def create_prompt_template(template):
    prompt = PromptTemplate(template = template, input_variables = ['context', 'question'])
    return prompt

def create_qa(llm, prompt, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = 'stuff',
        # k = 3 means the retriever will find maximum 3 related documents
        # retriever = db.as_retriever(search_kwargs = {'k': 5}, max_token_limit=1024),
        retriever = VectorDB().get_retriever(),
        return_source_documents = False,
        chain_type_kwargs = {'prompt': prompt}
    )
    return llm_chain

#### Response câu trả lời của model

In [ ]:
llm = load_model()

template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = create_prompt_template(template)

qa_chain = create_qa(llm, prompt, db)

#test
question = 'Sinh viên tốt nghiệp ngành công nghệ kỹ thuật Điện-Điện Tử (Điện công nghiệp) có thể làm những công việc gì?'
output = qa_chain.invoke({'query': question})
print(output)

#### Đánh giá chất lượng câu trả lời

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Giả sử bạn có câu tham chiếu lý tưởng (reference) cho câu hỏi đã đặt
reference = [
    ["các", "công", "việc", "liên", "quan", "đến", "điện", "công", "nghiệp", "và", "điện", "tử"],
    ["sinh", "viên", "tốt", "nghiệp", "có", "thể", "làm", "việc", "trong", "ngành", "điện", "công", "nghiệp"]
]

# Câu trả lời từ mô hình LLM của bạn
candidate = output.split()

# Tính điểm BLEU
smoothie = SmoothingFunction().method4
bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)

print(f"BLEU Score: {bleu_score:.4f}")